In [1]:
import torch
import pandas as pd

from rl_portfolio.environment import PortfolioOptimizationEnv
from rl_portfolio.algorithm import PolicyGradient
from rl_portfolio.architecture import EI3

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader

import logging
logging.getLogger('matplotlib.font_manager').disabled = True

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [2]:
TOP_BRL = [
    "VALE3.SA", "PETR4.SA", "ITUB4.SA", "BBDC4.SA",
    "BBAS3.SA", "RENT3.SA", "LREN3.SA", "PRIO3.SA",
    "WEGE3.SA", "ABEV3.SA"
]

TOP_NASDAQ = [
    "AAPL", "MU", "CSCO", "MSFT", "META", "QQQ",
    "CMCSA", "INTC", "HBAN", "NVDA"
]

bovespa_temporal = YahooDownloader(start_date = '2011-01-01',
                                end_date = '2022-12-31',
                                ticker_list = TOP_BRL).fetch_data()
bovespa_temporal

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Shape of DataFrame:  (29780, 8)


,date,open,high,low,close,volume,tic,day
0,2011-01-03,8.632311,8.728203,8.630313,5.265025,576145,ABEV3.SA,0
1,2011-01-03,15.750000,15.900000,15.690000,6.708954,6626800,BBAS3.SA,0
2,2011-01-03,11.809763,11.927362,11.724237,6.678930,10862336,BBDC4.SA,0
3,2011-01-03,18.031555,18.250118,17.963253,10.160485,10014663,ITUB4.SA,0
4,2011-01-03,8.422694,8.629907,8.422694,6.294306,3652542,LREN3.SA,0
...,...,...,...,...,...,...,...,...
29775,2022-12-29,24.990000,25.049999,24.430000,17.640100,57901800,PETR4.SA,3
29776,2022-12-29,37.250000,37.480000,36.619999,37.150330,7890300,PRIO3.SA,3
29777,2022-12-29,54.300869,55.776001,52.626392,51.326015,8786791,RENT3.SA,3
29778,2022-12-29,89.000000,89.949997,88.830002,80.014053,40110800,VALE3.SA,3


In [3]:
from rl_portfolio.data import GroupByScaler

df_portfolio = bovespa_temporal[["date", "tic", "close", "high", "low"]]

df_portfolio = GroupByScaler(columns = ["close", "high", "low"], by="tic").fit_transform(df_portfolio)

df_portfolio

,date,tic,close,high,low
0,2011-01-03,ABEV3.SA,0.272027,0.355383,0.359147
1,2011-01-03,BBAS3.SA,0.327830,0.568771,0.572419
2,2011-01-03,BBDC4.SA,0.287841,0.397479,0.399922
3,2011-01-03,ITUB4.SA,0.322621,0.458661,0.467793
4,2011-01-03,LREN3.SA,0.127001,0.157120,0.155466
...,...,...,...,...,...
29775,2022-12-29,PETR4.SA,0.694899,0.652514,0.662958
29776,2022-12-29,PRIO3.SA,0.857373,0.844144,0.878173
29777,2022-12-29,RENT3.SA,0.728887,0.754746,0.727218
29778,2022-12-29,VALE3.SA,0.916710,0.746783,0.770759


In [4]:
df_portfolio_train = df_portfolio[(df_portfolio["date"] >= "2011-01-01") & (df_portfolio["date"] < "2019-12-31")]
df_portfolio_test = df_portfolio[(df_portfolio["date"] >= "2019-10-16") & (df_portfolio["date"] < "2020-12-31")]

environment_train = PortfolioOptimizationEnv(
        df_portfolio_train,
        initial_amount=100000,
        state_normalization="by_last_close",
        comission_fee_pct=0.0025,
        time_window=50,
        time_format="%Y-%m-%d",
        features=["close", "high", "low"],
        print_metrics=False,
        plot_graphs=False,
        # comission_fee_model="trf_approx"
    )

environment_test = PortfolioOptimizationEnv(
        df_portfolio_test,
        initial_amount=100000,
        state_normalization="by_last_close",
        comission_fee_pct=0.0025,
        time_window=50,
        time_format="%Y-%m-%d",
        features=["close", "high", "low"],
        print_metrics=True,
        plot_graphs=True,
    )

In [5]:
algo = PolicyGradient(
    environment_train,
    # policy=EI3,
    validation_env=environment_test,
    sample_bias=0.5,
    batch_size=200,
    lr=5e-5,
    # parameter_noise=0.01,
    use_tensorboard=True,
    device=device
)

In [6]:
algo.train(episodes=120)

  1%|          | 1/120 [00:27<55:24, 27.94s/it]

Initial portfolio value:100000
Final portfolio value: 130531.015625
Final accumulative portfolio value: 1.30531015625
Maximum DrawDown: -0.5393334263433038
Sharpe ratio: 0.04756379578968361


  2%|▏         | 2/120 [00:55<54:21, 27.64s/it]

Initial portfolio value:100000
Final portfolio value: 141445.71875
Final accumulative portfolio value: 1.4144571875
Maximum DrawDown: -0.5882864265062817
Sharpe ratio: 0.05433248682797744


  2%|▎         | 3/120 [01:22<53:31, 27.45s/it]

...

Initial portfolio value:100000
Final portfolio value: 329273.9375
Final accumulative portfolio value: 3.292739375
Maximum DrawDown: -0.44701232783460476
Sharpe ratio: 0.12336609868331849


 99%|█████████▉| 119/120 [54:22<00:27, 27.43s/it]

Initial portfolio value:100000
Final portfolio value: 330341.53125
Final accumulative portfolio value: 3.3034153125
Maximum DrawDown: -0.443916223210609
Sharpe ratio: 0.12376876003961289


100%|██████████| 120/120 [54:50<00:00, 27.42s/it]

Initial portfolio value:100000
Final portfolio value: 332814.84375
Final accumulative portfolio value: 3.3281484375
Maximum DrawDown: -0.4416506169775004
Sharpe ratio: 0.12450743649457378


In [7]:
torch.save(algo.train_policy.state_dict(), "policy_BR.pt")

In [8]:
eiie_values = environment_test._asset_memory["final"]
eiie_values

[100000,
 107956.9,
 114202.59,
 116435.34,
 116586.21,
 114202.586,
 109827.586,
 112301.72,
 117129.31,
 124038.8,
 124612.07,
 124853.445,
 123737.055,
 129922.414,
 134931.03,
 139396.55,
 140392.25,
 137284.5,
 135112.08,
 143620.7,
 138068.98,
 142715.52,
 144616.38,
 144737.06,
 147241.38,
 144827.58,
 144495.69,
 145762.9,
 131702.58,
 137254.3,
 136228.44,
 135323.28,
 137284.48,
 135775.86,
 141657.22,
 153941.36,
 149758.48,
 157041.6,
 148145.75,
 143442.77,
 141397.2,
 153448.69,
 156485.42,
 158248.78,
 152501.34,
 143749.66,
 129805.78,
 141310.83,
 127549.74,
 101395.875,
 120051.46,
 98882.086,
 103027.3,
 88358.11,
 101094.32,
 96773.38,
 91866.7,
 105522.445,
 113476.68,
 114031.36,
 105393.83,
 106027.78,
 110861.63,
 113318.15,
 124360.516,
 110896.875,
 133209.45,
 137839.52,
 145571.81,
 141369.39,
 142293.94,
 142928.81,
 141741.7,
 135714.12,
 145739.48,
 147078.2,
 145831.19,
 141708.5,
 127084.89,
 134150.58,
 139108.62,
 145400.28,
 140560.64,
 139517.11,
 1

In [18]:
# uniform buy and hold
import numpy as np

environment_test.reset()
done = False

while not done:
    _, _, done, _, _ = environment_test.step(np.array([0] + [1/10] * 10))

Initial portfolio value:100000
Final portfolio value: 124423.1328125
Final accumulative portfolio value: 1.244231328125
Maximum DrawDown: -0.48198812549863623
Sharpe ratio: 0.04363526848832826


In [20]:
ubah_values = environment_test._asset_memory["final"]
ubah_values

[100000,
 102891.75,
 102706.75,
 102439.46,
 102186.91,
 101077.78,
 99700.766,
 99461.05,
 101126.72,
 101663.484,
 100852.94,
 101446.19,
 102846.984,
 103442.46,
 102269.86,
 103559.2,
 105267.58,
 104762.46,
 102350.2,
 104330.73,
 103423.1,
 103735.164,
 101797.81,
 102374.67,
 102904.82,
 103642.72,
 103108.516,
 102641.57,
 101285.45,
 104021.79,
 104821.12,
 103929.72,
 103067.25,
 103428.85,
 103438.94,
 104871.18,
 103935.87,
 103498.0,
 95922.234,
 93302.18,
 94155.234,
 96836.414,
 96051.19,
 97842.06,
 93863.47,
 89588.945,
 76895.76,
 82252.18,
 75917.81,
 63920.55,
 72791.57,
 61177.99,
 63308.7,
 55234.41,
 57866.113,
 57868.62,
 54529.855,
 60015.344,
 67116.15,
 69783.16,
 65483.438,
 65276.867,
 63582.273,
 61432.156,
 64209.156,
 61897.086,
 67867.63,
 71396.625,
 73826.945,
 71399.81,
 71754.34,
 72001.67,
 70543.82,
 68903.81,
 70681.234,
 70193.65,
 71400.16,
 70397.2,
 66516.69,
 68610.54,
 72996.86,
 74301.86,
 71220.805,
 69648.51,
 70466.82,
 69207.4,
 67646

In [21]:
# best performing stock
import numpy as np

environment_test.reset()
done = False

while not done:
    _, _, done, _, _ = environment_test.step(np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]))

Initial portfolio value:100000
Final portfolio value: 219731.40625
Final accumulative portfolio value: 2.1973140625
Maximum DrawDown: -0.4666771035168006
Sharpe ratio: 0.10559124967935572


In [22]:
best_stock_values = environment_test._asset_memory["final"]
best_stock_values

[100000,
 101419.23,
 100124.16,
 100383.164,
 101160.23,
 97332.55,
 95893.555,
 97850.6,
 98771.516,
 100728.51,
 101390.445,
 103232.34,
 105908.83,
 109160.91,
 107894.625,
 111952.56,
 114111.016,
 119233.77,
 117909.91,
 120902.95,
 121996.61,
 117909.914,
 113420.3,
 117046.54,
 116614.836,
 119061.09,
 123291.7,
 118284.06,
 119521.57,
 123896.07,
 128241.78,
 131148.5,
 131148.5,
 131752.88,
 128644.695,
 140501.9,
 136674.2,
 143322.3,
 135230.34,
 126971.555,
 124603.66,
 135692.31,
 138377.88,
 139937.25,
 134854.89,
 127115.88,
 114785.51,
 121022.914,
 113197.266,
 95293.63,
 99711.79,
 79151.47,
 82818.836,
 76437.06,
 82963.2,
 93474.375,
 101741.58,
 104227.336,
 107984.83,
 109690.15,
 102204.055,
 98330.95,
 97030.26,
 93735.22,
 98562.13,
 95064.805,
 96192.05,
 107811.44,
 115586.54,
 110701.78,
 113476.57,
 112436.04,
 109834.64,
 109863.56,
 111164.23,
 113014.06,
 118274.58,
 114430.39,
 111886.83,
 117349.664,
 120182.234,
 119661.97,
 115441.98,
 115152.99,
 1

In [23]:
# follow the loser: invest in the worst performing stock in training
import numpy as np

environment_test.reset()
done = False

while not done:
    _, _, done, _, _ = environment_test.step(np.array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]))

Initial portfolio value:100000
Final portfolio value: 211780.109375
Final accumulative portfolio value: 2.11780109375
Maximum DrawDown: -0.787909849982724
Sharpe ratio: 0.07947899879210203


In [24]:
follow_loser_values = environment_test._asset_memory["final"]
follow_loser_values

[100000,
 107956.9,
 114202.59,
 116435.34,
 116586.21,
 114202.586,
 109827.586,
 112301.72,
 117129.31,
 124038.8,
 124612.07,
 124853.445,
 123737.055,
 129922.414,
 134931.03,
 139396.55,
 140392.25,
 137284.5,
 135112.08,
 143620.7,
 138068.98,
 142715.52,
 144616.38,
 144737.06,
 147241.38,
 144827.58,
 144495.69,
 145762.9,
 131702.58,
 137254.3,
 136228.44,
 135323.28,
 137284.48,
 135775.86,
 141659.48,
 141961.19,
 144767.22,
 144706.89,
 123405.164,
 120086.2,
 118879.305,
 127629.31,
 126060.33,
 124702.58,
 119331.89,
 106931.03,
 67857.76,
 77030.17,
 70331.9,
 47340.52,
 56331.9,
 46767.24,
 44323.273,
 31228.445,
 35693.96,
 37413.793,
 35844.824,
 39435.34,
 52168.094,
 57116.37,
 54612.06,
 51293.098,
 50387.92,
 49724.13,
 58836.2,
 62849.13,
 77603.44,
 82974.125,
 84452.58,
 74525.86,
 70784.48,
 71478.44,
 70905.164,
 67887.92,
 66409.48,
 62155.164,
 59771.547,
 60193.957,
 55969.82,
 56573.266,
 65745.67,
 64900.844,
 61702.57,
 59530.156,
 63905.15,
 62728.43,


In [25]:
# follow the winner: invest in the best performing stock in training set
import numpy as np

environment_test.reset()
done = False

while not done:
    _, _, done, _, _ = environment_test.step(np.array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]))

Initial portfolio value:100000
Final portfolio value: 77903.015625
Final accumulative portfolio value: 0.77903015625
Maximum DrawDown: -0.5061583640682784
Sharpe ratio: -0.005289602162757053


In [26]:
follow_winner_values = environment_test._asset_memory["final"]
follow_winner_values

[100000,
 102075.54,
 101365.44,
 101276.66,
 101223.41,
 101880.25,
 101099.15,
 101418.7,
 103619.96,
 102306.305,
 101631.72,
 102448.336,
 105128.92,
 105625.97,
 104649.61,
 106247.31,
 108058.07,
 106815.38,
 104525.36,
 105927.79,
 106513.59,
 105093.414,
 101986.78,
 102572.58,
 101010.39,
 101542.984,
 98028.016,
 98649.34,
 97566.47,
 100282.555,
 100566.59,
 101738.25,
 100211.55,
 101542.984,
 100921.64,
 100318.07,
 98578.31,
 99412.68,
 93465.66,
 90625.31,
 94619.57,
 94086.99,
 94672.82,
 96838.6,
 91956.73,
 90145.984,
 81660.414,
 89205.125,
 83595.41,
 76707.52,
 83045.09,
 65505.848,
 68417.23,
 56807.254,
 65328.34,
 61208.027,
 53363.574,
 58682.145,
 65459.33,
 68821.24,
 64036.3,
 63484.88,
 59696.062,
 57365.85,
 57170.188,
 55071.223,
 61812.824,
 65815.086,
 67149.19,
 65067.992,
 64107.47,
 64143.027,
 63591.613,
 63413.734,
 66704.48,
 69372.664,
 72805.74,
 70173.13,
 67006.9,
 68767.9,
 71542.79,
 72165.375,
 68320.98,
 66220.97,
 64690.465,
 63409.11,
 5

In [34]:
import matplotlib.pyplot as plt

plt.plot(eiie_values, "tab:blue", label="EIIE Conv")
plt.plot(best_stock_values, "tab:green", label="Best Stock")
plt.plot(follow_loser_values, "tab:red", label="Follow the Loser")
plt.plot(ubah_values, "tab:orange", label="Uniform Buy & Hold")
plt.plot(follow_winner_values, "tab:purple", label="Follow the Winner")
plt.title("Portfolio Value Over Time in 2020")
plt.legend()
plt.xticks(
    np.round(np.linspace(0, len(eiie_values) - 1, 10))
)
plt.xlabel("Trading Days")
plt.ylabel("Portfolio Value")
plt.savefig("comparison.png", bbox_inches="tight")
plt.close()